<a href="https://colab.research.google.com/github/soumabkargit/Extract_Text_Image/blob/master/TesseractMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tesseract installation**

[https://github.com/tesseract-ocr/tesseract/wiki](https://)


In [101]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install tesseract-ocr-eng
!pip install remotezip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-eng is already the newest version (4.00~git24-0e00fe6-1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


##Importing all useful dependencies

In [102]:
from matplotlib.image import imread
import numpy as np
from matplotlib import pyplot as plt
import os
import pytesseract
from pytesseract import Output
import matplotlib.patches as patches
from PIL import Image
import re
from pathlib import Path
from shutil import copyfile
from remotezip import RemoteZip
import json
import csv
# Technically not necessary in newest versions of jupyter
%matplotlib inline

In [103]:
def hasNumbers(inputString):
  return any(char.isdigit() for char in inputString)

def extract_total(array):
  max_array = 0
  temp = []
  if len(array) == 1 :
    temp = [float(s) for s in array[0].split() if s.isdigit()]
    max_array = temp[0]
    print(max_array)
  elif len(array) > 1 :
    for i in range(len(array)):
      temp = max([int(s) for s in array[i].split() if s.isdigit()])
      if max_array < temp[0]:
        max_array = temp[0]    
  else :
    print('empty array')
  return max_array

def extract_amount(regex_amount, string):
  return re.search(regex_amount, string, re.IGNORECASE)


def createFolder(path):
  try:
    os.mkdir(path)
  except OSError:
    print ("Creation of the directory %s failed" % path)
  else:
    print ("Successfully created the directory %s " % path)

def extracted(url, data):
  with RemoteZip(url) as zip:
    zip.extractall(path=data)
  
def extracted_text_split(imageName, lang):
  im = np.array(Image.open(imageName), dtype=np.uint8)
  extracted_text = pytesseract.image_to_string(im, lang)
  text_split_line = extracted_text.splitlines()
  return text_split_line
  
def regexLine(regex, line):
  return re.search(regex, line, re.IGNORECASE)
    
def copy_file_folder(file_name, path):
  copyfile(file_name, path)


In [104]:
url = 'https://expressexpense.com/large-receipt-image-dataset-SRD.zip'
lang = 'eng'
image_extract_path = "image_extract"
image_NO_extract_path = "image_NO_extract"
image_NO_read_text_path = "image_NO_read_text"
data = "data"
list_regex = []
regex_1 = r'^TOTAL(.)*(\:)*(.)*(\d)*'
regex_2 = r'^BALANCE(.)*(\:)*(.)*(\d)*'
regex_3 = r'^AMOUNT(.)*(\:)*(.)*(\d)*'
regex_amount = r'(\d)*'
list_regex.append(regex_1)
list_regex.append(regex_2)
list_regex.append(regex_3)
list_good_files = []
list_bad_files = []
list_word_total = []
candidat_good_total = []
candidat_bad_total = []
list_candidat_word_total = []
list_word_total_file = []
list_invoices = {}
receipt_ocr = {}
total_invoice_good_info = {}
total_invoice_bad_info = {}
all_total_invoice_info = {}

In [ ]:
createFolder(image_extract_path)
createFolder(image_NO_extract_path)
createFolder(image_NO_read_text_path)
createFolder(data)

In [87]:
extracted(url, data)

In [106]:
list_files = os.listdir(data)

##Dataset

https://expressexpense.com/large-receipt-image-dataset-SRD.zip

## Reading of all invoice images and extraction of the total amount.

##Sort extracted and non-extracted images into respective folders.

In [107]:
for image_filename in list_files:

  im = data+'/'+image_filename
  text_split_text = extracted_text_split(im, lang)
  
  for line in text_split_text:
    for regex in list_regex:
      if regexLine(regex, line):
        list_word_total_file.append(line)     

  if len(list_word_total_file) == 0:
    copy_file_folder(data + '/' + image_filename, image_NO_read_text_path + '/' + image_filename)
  
  for el in list_word_total_file:
    if hasNumbers(el):
      candidat_good_total.append(el)
    else:
      candidat_bad_total.append(el)

  if len(candidat_good_total) > 0:
    copyfile(data + '/' + image_filename, image_extract_path + '/' + image_filename)
    total_invoice_good_info[image_filename] = candidat_good_total
    list_good_files.append(image_filename)
  elif len(candidat_bad_total) > 0 : 
    copy_file_folder(data + '/' + image_filename, image_NO_extract_path + '/' + image_filename)
    total_invoice_bad_info[image_filename] = candidat_bad_total
    list_bad_files.append(image_filename)
  
  list_word_total_file = []
  candidat_good_total = []
  candidat_bad_total = []

total_invoice_good_info

{'1002-receipt.jpg': ['Total 161'],
 '1004-receipt.jpg': ['TOTAL. $1IS.03'],
 '1005-receipt.jpg': ['Balance Due 179.94'],
 '1007-receipt.jpg': ['Total $143.71'],
 '1008-receipt.jpg': ['TOTAL: $24.47'],
 '1012-receipt.jpg': ['Total 24.36', 'Total $33 26'],
 '1016-receipt.jpg': ['Total $35.52'],
 '1023-receipt.jpg': ['Total 4 item(s)'],
 '1024-receipt.jpg': ['AMOUNT : 31.12', 'TOTAL : 33.92'],
 '1027-receipt.jpg': ['Total $41.31'],
 '1031-receipt.jpg': ['Total Due: 44.35'],
 '1035-receipt.jpg': ['total: $24.25'],
 '1037-receipt.jpg': ['Total 164.89'],
 '1040-receipt.jpg': ['Total $28.04'],
 '1048-receipt.jpg': ['Total 91.45', 'Balance Due 91.45'],
 '1049-receipt.jpg': ['Total Tax: $3.49', 'Total: $77.49'],
 '1050-receipt.jpg': ['Total Tax 1.74'],
 '1054-receipt.jpg': ['Total $56.50'],
 '1057-receipt.jpg': ['Total $9.58'],
 '1077-receipt.jpg': ['Total $69.79'],
 '1079-receipt.jpg': ['TOTAL 235.19'],
 '1080-receipt.jpg': ['TOTAL: 92.22'],
 '1081-receipt.jpg': ['Total $9.97'],
 '1085-receip

#References

https://nanonets.com/blog/receipt-ocr/?fbclid=IwAR0H4VkHhpimqKIgEUQOUCOJS1mtk6vQREcv3MMiqJ2NpxG9LHZnsHM-aeE


https://www.datacamp.com/community/tutorials/zip-file